# Example of the Junior Notebook

In this Notebook, we present to ways to train the junior model. Either, with the simple Junior model or with an
advanced model that can be obtained, if some hyper-parameter tuning was conducted

In [ ]:
import os
import logging
from pathlib import Path
import pickle
import json
import grid2op
import numpy as np
from lightsim2grid import LightSimBackend

import tensorflow as tf

logging.basicConfig(level=logging.INFO)
example_path = os.getcwd()

Again check the environment

In [ ]:
backend = LightSimBackend()
env = grid2op.make("l2rpn_neurips_2020_track1_small", backend=backend)
env.set_id(1)
env.reset()
env.chronics_handler.get_name()
obs = env.get_obs()

## Simple Junior

Let's begin with the simple junior for training

In [ ]:
from curriculumagent.junior.junior_student import train, validate

In [ ]:
action_set = Path(example_path) / "example_teacher_results.npy"
model_path = Path(example_path) / "junior_model"
data_path = Path(example_path) / "junior_data"

train(run_name="example",
      dataset_path=data_path,
      target_model_path=model_path,
      action_space_file=action_set,
      dataset_name='junior_dataset',
      epochs=100, seed=42)

In [ ]:
validate(checkpoint_path=model_path,
         dataset_path=data_path,
         dataset_name='junior_dataset',
         action_space_file=action_set)

## Advanced Junior

The second option is a more advanced junior agent, that has selected hyper-parameters. You can gather these
hyperparameters by running a NNI or ```ray.tune``` experiment. The hyper-parameters need then to be supplied via a
dictionary. Note that it is also possible to add a scaler, e.g., a ```MinMaxScaler``` to the Junior. We advise for this \
        option, given that neural networks require values around [-1,1].

Note, for simplification we only load the best parameters into the Notebook.

In [ ]:
with open('junior_best_params.json') as json_file:
    best_params = json.load(json_file)

best_params["epochs"] = 1000
best_params["initializer"] = tf.keras.initializers.Orthogonal()

best_params

In [ ]:
from curriculumagent.junior.hyper_parameter_search.advanced_junior_student import AdvancedJunior
from curriculumagent.junior.junior_student import load_dataset
from sklearn.preprocessing import MinMaxScaler

Due to the fact that we also want to show the application of a scaler, we additionally show the whole pipeline of the   Junior agent. Therefore, we first fit the data with a MinMaxScaler:


In [ ]:
s_train, a_train, s_validate, a_validate, _, _ = load_dataset(data_path, "example")
scaler = MinMaxScaler()
s_tr_t = scaler.fit_transform(s_train)
s_val_t = scaler.transform(s_validate)

In [ ]:
# Pickle the scaler for later
with open('scaler_junior.pkl', "wb") as fp:
    pickle.dump(scaler, fp)

In [ ]:
junior = AdvancedJunior(config=best_params,
                        trainset_size=len(a_train),
                        # Optimally, you know the number of actions:
                        num_actions=np.max(a_train) + 1,
                        seed=42)
junior.model.summary()

In [ ]:
history = junior.train(log_dir=model_path / "hyperparam",
                       ckpt_dir=model_path / "hyperparam",
                       patience=5,
                       x_train=s_tr_t, y_train=a_train,
                       x_validate=s_val_t, y_validate=a_validate)

In [ ]:
junior.model.save(model_path / "hyperparam")